## 1. Configuration initiale

Importation des bibliothèques nécessaires et définition des paramètres de connexion à la base de données.

In [1]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from pathlib import Path
import time

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

Dossier des données transformées: C:\Users\Saint\IdeaProjects\ETL_Simple_Project\ETL_Projet_Pandemies\donnees\transformees


## 2. Configuration de la connexion à la base de données

Définition des paramètres de connexion à la base de données MySQL.

In [2]:
# Paramètres de connexion à la base de données
config_db = {
    "host": "localhost",
    "user": "root",
    "password": "",  # Laisser vide si aucun mot de passe
    "database": "epiviz",
    "port": 3306
}

print("Paramètres de connexion à la base de données:")
for cle, valeur in config_db.items():
    if cle != "password":
        print(f"- {cle}: {valeur}")
    else:
        print(f"- {cle}: {'(aucun)' if valeur == '' else '******'}")

Paramètres de connexion à la base de données:
- host: localhost
- user: root
- password: (aucun)
- database: epiviz
- port: 3306


## 3. Fonction de connexion à la base de données

Définition d'une fonction pour établir une connexion à la base de données MySQL.

In [3]:
def connecter_mysql(config=None):
    """
    Établit une connexion à la base de données MySQL.
    
    Args:
        config (dict): Dictionnaire contenant les paramètres de connexion.
        
    Returns:
        tuple: (connexion, curseur) si la connexion est établie, (None, None) sinon.
    """
    if config is None:
        config = config_db.copy()
    
    try:
        # Connexion à la base de données
        connexion = mysql.connector.connect(**config)
        curseur = connexion.cursor()
        print(f"Connexion à la base de données '{config['database']}' établie avec succès.")
        return connexion, curseur
    
    except Error as e:
        print(f"Erreur lors de la connexion à la base de données: {e}")
        return None, None

## 4. Chargement des données transformées

Chargement des données transformées depuis les fichiers CSV.

In [ ]:
# Chargement des données transformées
print("Chargement des données transformées...")

tables = {}

try:
    # Chargement du fichier de données final
    fichier_donnees_final = dossier_donnees_transformees / "donnees_final.csv"
    if fichier_donnees_final.exists():
        donnees_final = pd.read_csv(fichier_donnees_final, parse_dates=["date"])
        print(f"Fichier donnees_final.csv chargé: {donnees_final.shape[0]} lignes, {donnees_final.shape[1]} colonnes")
        
        # Création de la table calendar à partir des dates uniques
        print("\nCréation de la table calendar...")
        tables["calendar"] = donnees_final[["date"]].drop_duplicates().reset_index(drop=True)
        # Convertir la date en valeur entière (timestamp) pour correspondre à date_value
        tables["calendar"]["date_value"] = tables["calendar"]["date"].apply(lambda x: int(x.timestamp()))
        tables["calendar"]["id"] = tables["calendar"].index + 1  # Création d'un ID auto-incrémenté
        tables["calendar"] = tables["calendar"][["id", "date_value"]]
        print(f"Table calendar créée: {tables['calendar'].shape[0]} lignes, {tables['calendar'].shape[1]} colonnes")
        
        # Création de la table location à partir des pays uniques
        print("\nCréation de la table location...")
        tables["location"] = donnees_final[["pays"]].drop_duplicates().reset_index(drop=True)
        tables["location"]["continent"] = "Unknown"  # Valeur par défaut pour continent
        tables["location"]["id"] = tables["location"].index + 1  # Création d'un ID auto-incrémenté
        tables["location"] = tables["location"][["id", "pays", "continent"]]
        tables["location"] = tables["location"].rename(columns={"pays": "country"})
        print(f"Table location créée: {tables['location'].shape[0]} lignes, {tables['location'].shape[1]} colonnes")
        
        # Création de la table pandemie à partir des types de pandémie uniques
        print("\nCréation de la table pandemie...")
        tables["pandemie"] = donnees_final[["type_pandemie"]].drop_duplicates().reset_index(drop=True)
        tables["pandemie"]["id"] = tables["pandemie"].index + 1  # Création d'un ID auto-incrémenté
        tables["pandemie"] = tables["pandemie"][["id", "type_pandemie"]]
        tables["pandemie"] = tables["pandemie"].rename(columns={"type_pandemie": "type"})
        print(f"Table pandemie créée: {tables['pandemie'].shape[0]} lignes, {tables['pandemie'].shape[1]} colonnes")
        
        # Création de la table data en joignant les données avec les tables de référence
        print("\nCréation de la table data...")
        # Création d'un DataFrame temporaire pour les jointures
        donnees_temp = donnees_final.copy()
        
        # Jointure avec la table calendar pour obtenir id_calendar
        donnees_temp = donnees_temp.merge(
            tables["calendar"][["id", "date_value"]],
            left_on=donnees_temp["date"].apply(lambda x: int(x.timestamp())),
            right_on="date_value",
            how="left"
        )
        donnees_temp = donnees_temp.rename(columns={"id": "id_calendar"})
        
        # Jointure avec la table location pour obtenir id_location
        donnees_temp = donnees_temp.merge(
            tables["location"].rename(columns={"country": "pays"})[["id", "pays"]],
            on="pays",
            how="left"
        )
        donnees_temp = donnees_temp.rename(columns={"id": "id_location"})
        
        # Jointure avec la table pandemie pour obtenir id_pandemie
        donnees_temp = donnees_temp.merge(
            tables["pandemie"].rename(columns={"type": "type_pandemie"})[["id", "type_pandemie"]],
            on="type_pandemie",
            how="left"
        )
        donnees_temp = donnees_temp.rename(columns={"id": "id_pandemie"})
        
        # Création de la table data finale
        tables["data"] = pd.DataFrame()
        tables["data"]["id"] = range(1, len(donnees_temp) + 1)
        tables["data"]["total_cases"] = donnees_temp["cas_confirmes"]
        tables["data"]["total_deaths"] = donnees_temp["deces"]
        tables["data"]["new_cases"] = 0  # Valeur par défaut, à calculer si nécessaire
        tables["data"]["new_deaths"] = 0  # Valeur par défaut, à calculer si nécessaire
        tables["data"]["id_location"] = donnees_temp["id_location"]
        tables["data"]["id_pandemie"] = donnees_temp["id_pandemie"]
        tables["data"]["id_calendar"] = donnees_temp["id_calendar"]
        
        print(f"Table data créée: {tables['data'].shape[0]} lignes, {tables['data'].shape[1]} colonnes")
        
    else:
        print(f"Impossible de charger le fichier donnees_final.csv: fichier non trouvé")
        
except Exception as e:
    print(f"Erreur lors du chargement des données: {e}")
    
print("\nChargement et transformation des données terminés avec succès!")

## 5. Préparation des données pour le chargement

Préparation des données pour le chargement dans la base de données MySQL.

In [ ]:
def vider_tables(connexion, curseur, tables_a_vider=None):
    """
    Vide les tables spécifiées dans la base de données.
    
    Args:
        connexion: Connexion à la base de données.
        curseur: Curseur de la base de données.
        tables_a_vider (list): Liste des noms des tables à vider. Si None, toutes les tables sont vidées.
        
    Returns:
        bool: True si l'opération a réussi, False sinon.
    """
    try:
        # Désactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        # Liste des tables à vider
        if tables_a_vider is None:
            tables_a_vider = ["data", "calendar", "location", "pandemie"]
        
        # Vidage des tables
        print("Vidage des tables...")
        for table in tables_a_vider:
            curseur.execute(f"TRUNCATE TABLE {table};")
            print(f"Table {table} vidée.")
        
        # Réactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 1;")
        
        # Validation des modifications
        connexion.commit()
        print("\nOpération de vidage terminée.")
        return True
    
    except Error as e:
        print(f"Erreur lors du vidage des tables: {e}")
        return False

## 6. Fonction de vidage des tables

Définition d'une fonction pour vider les tables de la base de données.

In [ ]:
def charger_donnees(connexion, curseur, table_nom, table_df, taille_lot=1000):
    """
    Charge les données d'un DataFrame dans une table de la base de données.
    
    Args:
        connexion: Connexion à la base de données.
        curseur: Curseur de la base de données.
        table_nom (str): Nom de la table dans laquelle charger les données.
        table_df (DataFrame): DataFrame contenant les données à charger.
        taille_lot (int): Nombre de lignes à charger par lot.
        
    Returns:
        bool: True si l'opération a réussi, False sinon.
    """
    try:
        # Récupération des noms des colonnes
        colonnes = list(table_df.columns)
        print(f"Colonnes de la table {table_nom}: {colonnes}")
        
        # Vérification des valeurs nulles
        for colonne in colonnes:
            null_count = table_df[colonne].isnull().sum()
            if null_count > 0:
                print(f"ATTENTION: La colonne {colonne} contient {null_count} valeurs nulles!")
                
                # Pour les colonnes numériques, remplacer les valeurs nulles par 0
                if pd.api.types.is_numeric_dtype(table_df[colonne]):
                    table_df[colonne] = table_df[colonne].fillna(0)
                    print(f"  -> Valeurs nulles dans la colonne {colonne} remplacées par 0")
                # Pour les colonnes de type chaîne, remplacer les valeurs nulles par une chaîne vide
                elif pd.api.types.is_string_dtype(table_df[colonne]):
                    table_df[colonne] = table_df[colonne].fillna("")
                    print(f"  -> Valeurs nulles dans la colonne {colonne} remplacées par une chaîne vide")
        
        # Préparation de la requête d'insertion
        placeholders = ", ".join(["%s"] * len(colonnes))
        requete = f"INSERT INTO {table_nom} ({', '.join(colonnes)}) VALUES ({placeholders})"
        
        # Conversion des données en liste de tuples
        donnees = [tuple(x) for x in table_df.to_numpy()]
        
        # Affichage des premières lignes pour débogage
        print(f"Exemple de données à charger dans {table_nom} (3 premières lignes):")
        for i in range(min(3, len(donnees))):
            print(donnees[i])
        
        # Chargement des données par lots
        print(f"Chargement de {len(donnees)} lignes dans la table {table_nom}...")
        
        for i in range(0, len(donnees), taille_lot):
            lot = donnees[i:i + taille_lot]
            try:
                curseur.executemany(requete, lot)
                connexion.commit()
                print(f"Lot {i // taille_lot + 1}/{(len(donnees) + taille_lot - 1) // taille_lot} chargé: {len(lot)} lignes")
            except Error as e:
                print(f"Erreur lors du chargement du lot {i // taille_lot + 1}: {e}")
                # Afficher la première ligne qui pose problème
                if len(lot) > 0:
                    print(f"Première ligne du lot problématique: {lot[0]}")
                raise
        
        print(f"Données chargées dans la table {table_nom} avec succès.")
        return True
    
    except Error as e:
        print(f"Erreur lors du chargement des données dans la table {table_nom}: {e}")
        return False

## 7. Fonction de chargement des données

Définition d'une fonction pour charger les données dans la base de données.

In [ ]:
def creer_base_donnees(config):
    """
    Crée la base de données si elle n'existe pas.
    
    Args:
        config (dict): Dictionnaire contenant les paramètres de connexion.
        
    Returns:
        bool: True si l'opération a réussi, False sinon.
    """
    try:
        # Copie de la configuration sans la base de données
        config_sans_db = config.copy()
        nom_db = config_sans_db.pop('database')
        
        # Connexion à MySQL sans spécifier de base de données
        connexion = mysql.connector.connect(**config_sans_db)
        curseur = connexion.cursor()
        
        # Création de la base de données si elle n'existe pas
        curseur.execute(f"CREATE DATABASE IF NOT EXISTS {nom_db}")
        print(f"Base de données '{nom_db}' créée ou déjà existante.")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        
        return True
        
    except Error as e:
        print(f"Erreur lors de la création de la base de données: {e}")
        return False

## 8. Chargement des données dans la base de données

Chargement des données dans la base de données MySQL.

In [ ]:
# Importations nécessaires
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np
from pathlib import Path

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Paramètres de connexion à la base de données
config_db = {
    "host": "localhost",
    "user": "root",
    "password": "",  # Laisser vide si aucun mot de passe
    "database": "epiviz",
    "port": 3306
}

# Fonction de connexion à la base de données
def connecter_mysql(config=None):
    if config is None:
        config = config_db.copy()
    
    try:
        connexion = mysql.connector.connect(**config)
        curseur = connexion.cursor()
        print(f"Connexion à la base de données '{config['database']}' établie avec succès.")
        return connexion, curseur
    
    except Error as e:
        print(f"Erreur lors de la connexion à la base de données: {e}")
        return None, None

# Fonction pour créer la base de données
def creer_base_donnees(config):
    try:
        # Copie de la configuration sans la base de données
        config_sans_db = config.copy()
        nom_db = config_sans_db.pop('database')
        
        # Connexion à MySQL sans spécifier de base de données
        connexion = mysql.connector.connect(**config_sans_db)
        curseur = connexion.cursor()
        
        # Création de la base de données si elle n'existe pas
        curseur.execute(f"DROP DATABASE IF EXISTS {nom_db}")
        curseur.execute(f"CREATE DATABASE IF NOT EXISTS {nom_db} DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci")
        print(f"Base de données '{nom_db}' créée avec succès.")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        
        return True
        
    except Error as e:
        print(f"Erreur lors de la création de la base de données: {e}")
        return False

# Fonction pour charger les données
def charger_donnees(connexion, curseur, table_nom, table_df, taille_lot=1000):
    try:
        # Récupération des noms des colonnes
        colonnes = list(table_df.columns)
        print(f"Colonnes de la table {table_nom}: {colonnes}")
        
        # Vérification des valeurs nulles
        for colonne in colonnes:
            null_count = table_df[colonne].isnull().sum()
            if null_count > 0:
                print(f"ATTENTION: La colonne {colonne} contient {null_count} valeurs nulles!")
        
        # Préparation de la requête d'insertion
        placeholders = ", ".join(["%s"] * len(colonnes))
        requete = f"INSERT INTO {table_nom} ({', '.join(colonnes)}) VALUES ({placeholders})"
        
        # Conversion des données en liste de tuples avec conversion explicite des types numpy en types Python
        donnees = []
        for row in table_df.itertuples(index=False):
            # Convertir chaque valeur numpy en type Python standard
            python_row = []
            for val in row:
                if isinstance(val, np.integer):
                    val = int(val)
                elif isinstance(val, np.floating):
                    val = float(val)
                elif isinstance(val, np.ndarray):
                    val = val.tolist()
                python_row.append(val)
            donnees.append(tuple(python_row))
        
        # Affichage des premières lignes pour débogage
        print(f"Exemple de données à charger dans {table_nom} (3 premières lignes):")
        for i in range(min(3, len(donnees))):
            print(donnees[i])
        
        # Chargement des données par lots
        print(f"Chargement de {len(donnees)} lignes dans la table {table_nom}...")
        
        for i in range(0, len(donnees), taille_lot):
            lot = donnees[i:i + taille_lot]
            try:
                curseur.executemany(requete, lot)
                connexion.commit()
                print(f"Lot {i // taille_lot + 1}/{(len(donnees) + taille_lot - 1) // taille_lot} chargé: {len(lot)} lignes")
            except Error as e:
                print(f"Erreur lors du chargement du lot {i // taille_lot + 1}: {e}")
                if len(lot) > 0:
                    print(f"Première ligne du lot problématique: {lot[0]}")
                raise
        
        print(f"Données chargées dans la table {table_nom} avec succès.")
        return True
    
    except Error as e:
        print(f"Erreur lors du chargement des données dans la table {table_nom}: {e}")
        return False

# Création et configuration de la base de données
try:
    # Création de la base de données
    creer_base_donnees(config_db)
    
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Désactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 0;")
        
        print("Création des tables...")
        
        # Table calendar
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS calendar (
          id INT NOT NULL AUTO_INCREMENT,
          date_value INT DEFAULT NULL,
          PRIMARY KEY (id)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
        """)
        print("Table calendar créée.")
        
        # Table location
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS location (
          id INT NOT NULL AUTO_INCREMENT,
          country CHAR(50) NOT NULL DEFAULT '0',
          continent CHAR(50) NOT NULL DEFAULT '0',
          PRIMARY KEY (id)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
        """)
        print("Table location créée.")
        
        # Table pandemie
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS pandemie (
          id INT NOT NULL AUTO_INCREMENT,
          type CHAR(50) NOT NULL,
          PRIMARY KEY (id)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
        """)
        print("Table pandemie créée.")
        
        # Table data
        curseur.execute("""
        CREATE TABLE IF NOT EXISTS data (
          id INT NOT NULL AUTO_INCREMENT,
          total_cases INT(30) NOT NULL,
          total_deaths INT(30) NOT NULL,
          new_cases INT(30) NOT NULL,
          new_deaths INT(30) NOT NULL,
          id_location INT(30) NOT NULL,
          id_pandemie INT(30) NOT NULL,
          id_calendar INT(30) NOT NULL,
          PRIMARY KEY (id),
          KEY id_pandemie (id_pandemie),
          KEY id_location (id_location),
          KEY id_calendar (id_calendar),
          CONSTRAINT id_calendar FOREIGN KEY (id_calendar) REFERENCES calendar (id) ON DELETE NO ACTION ON UPDATE NO ACTION,
          CONSTRAINT id_location FOREIGN KEY (id_location) REFERENCES location (id) ON DELETE NO ACTION ON UPDATE NO ACTION,
          CONSTRAINT id_pandemie FOREIGN KEY (id_pandemie) REFERENCES pandemie (id) ON DELETE NO ACTION ON UPDATE NO ACTION
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
        """)
        print("Table data créée.")
        
        # Réactivation des contraintes de clé étrangère
        curseur.execute("SET FOREIGN_KEY_CHECKS = 1;")
        connexion.commit()
        
        print("Toutes les tables ont été créées avec succès.")
        
        # Chargement des données transformées à partir du fichier donnees_final.csv
        print("\nChargement des données transformées...")
        
        tables = {}
        
        # Chargement du fichier de données final
        fichier_donnees_final = dossier_donnees_transformees / "donnees_final.csv"
        if fichier_donnees_final.exists():
            donnees_final = pd.read_csv(fichier_donnees_final, parse_dates=["date"])
            print(f"Fichier donnees_final.csv chargé: {donnees_final.shape[0]} lignes, {donnees_final.shape[1]} colonnes")
            
            # Création de la table calendar à partir des dates uniques
            print("\nCréation de la table calendar...")
            tables["calendar"] = donnees_final[["date"]].drop_duplicates().reset_index(drop=True)
            # Convertir la date en valeur entière (timestamp) pour correspondre à date_value
            tables["calendar"]["date_value"] = tables["calendar"]["date"].apply(lambda x: int(x.timestamp()))
            tables["calendar"]["id"] = tables["calendar"].index + 1  # Création d'un ID auto-incrémenté
            tables["calendar"] = tables["calendar"][["id", "date_value"]]
            print(f"Table calendar créée: {tables['calendar'].shape[0]} lignes, {tables['calendar'].shape[1]} colonnes")
            
            # Création de la table location à partir des pays uniques
            print("\nCréation de la table location...")
            tables["location"] = donnees_final[["pays"]].drop_duplicates().reset_index(drop=True)
            tables["location"]["continent"] = "Unknown"  # Valeur par défaut pour continent
            tables["location"]["id"] = tables["location"].index + 1  # Création d'un ID auto-incrémenté
            tables["location"] = tables["location"][["id", "pays", "continent"]]
            tables["location"] = tables["location"].rename(columns={"pays": "country"})
            print(f"Table location créée: {tables['location'].shape[0]} lignes, {tables['location'].shape[1]} colonnes")
            
            # Création de la table pandemie à partir des types de pandémie uniques
            print("\nCréation de la table pandemie...")
            tables["pandemie"] = donnees_final[["type_pandemie"]].drop_duplicates().reset_index(drop=True)
            tables["pandemie"]["id"] = tables["pandemie"].index + 1  # Création d'un ID auto-incrémenté
            tables["pandemie"] = tables["pandemie"][["id", "type_pandemie"]]
            tables["pandemie"] = tables["pandemie"].rename(columns={"type_pandemie": "type"})
            print(f"Table pandemie créée: {tables['pandemie'].shape[0]} lignes, {tables['pandemie'].shape[1]} colonnes")
            
            # Création de la table data en joignant les données avec les tables de référence
            print("\nCréation de la table data...")
            # Création d'un DataFrame temporaire pour les jointures
            donnees_temp = donnees_final.copy()
            
            # Jointure avec la table calendar pour obtenir id_calendar
            donnees_temp = donnees_temp.merge(
                tables["calendar"][["id", "date_value"]],
                left_on=donnees_temp["date"].apply(lambda x: int(x.timestamp())),
                right_on="date_value",
                how="left"
            )
            donnees_temp = donnees_temp.rename(columns={"id": "id_calendar"})
            
            # Jointure avec la table location pour obtenir id_location
            donnees_temp = donnees_temp.merge(
                tables["location"].rename(columns={"country": "pays"})[["id", "pays"]],
                on="pays",
                how="left"
            )
            donnees_temp = donnees_temp.rename(columns={"id": "id_location"})
            
            # Jointure avec la table pandemie pour obtenir id_pandemie
            donnees_temp = donnees_temp.merge(
                tables["pandemie"].rename(columns={"type": "type_pandemie"})[["id", "type_pandemie"]],
                on="type_pandemie",
                how="left"
            )
            donnees_temp = donnees_temp.rename(columns={"id": "id_pandemie"})
            
            # Création de la table data finale
            tables["data"] = pd.DataFrame()
            tables["data"]["id"] = range(1, len(donnees_temp) + 1)
            tables["data"]["total_cases"] = donnees_temp["cas_confirmes"]
            tables["data"]["total_deaths"] = donnees_temp["deces"]
            tables["data"]["new_cases"] = 0  # Valeur par défaut
            tables["data"]["new_deaths"] = 0  # Valeur par défaut
            tables["data"]["id_location"] = donnees_temp["id_location"]
            tables["data"]["id_pandemie"] = donnees_temp["id_pandemie"]
            tables["data"]["id_calendar"] = donnees_temp["id_calendar"]
            
            print(f"Table data créée: {tables['data'].shape[0]} lignes, {tables['data'].shape[1]} colonnes")
            
            # Chargement des données dans l'ordre des dépendances
            ordre_chargement = ["calendar", "location", "pandemie", "data"]
            
            for table_nom in ordre_chargement:
                if table_nom in tables:
                    print(f"\nChargement des données dans la table {table_nom}...")
                    
                    # Remplacer les valeurs NaN par des valeurs appropriées selon le type de colonne
                    for colonne in tables[table_nom].columns:
                        if pd.api.types.is_numeric_dtype(tables[table_nom][colonne]):
                            # Remplacer les valeurs NaN par 0 pour les colonnes numériques
                            tables[table_nom][colonne] = tables[table_nom][colonne].fillna(0)
                        elif pd.api.types.is_object_dtype(tables[table_nom][colonne]):
                            # Remplacer les valeurs NaN par une chaîne vide pour les colonnes texte
                            tables[table_nom][colonne] = tables[table_nom][colonne].fillna('')
                    
                    # Chargement des données
                    charger_donnees(connexion, curseur, table_nom, tables[table_nom])
            
            print("\nToutes les données ont été chargées avec succès.")
        else:
            print(f"Impossible de charger le fichier donnees_final.csv: fichier non trouvé")
        
except Exception as e:
    print(f"Erreur lors du chargement des données dans la base de données: {e}")
    
finally:
    if 'connexion' in locals() and connexion is not None and connexion.is_connected():
        curseur.close()
        connexion.close()
        print("Connexion à la base de données fermée.")